In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import seaborn as sns
from scipy.stats import mode
import time

In [3]:
data = sns.load_dataset('titanic')

In [4]:
df = data.drop(['deck','class','who','adult_male','embark_town','alive','alone'],axis=1)

In [5]:
df['age'] = df['age'].fillna(df.groupby('sex')['age'].transform('mean'))

In [6]:
df['fam'] = df['parch'] + df['sibsp']

In [7]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,fam
0,0,3,male,22.0,1,0,7.2500,S,1
1,1,1,female,38.0,1,0,71.2833,C,1
2,1,3,female,26.0,0,0,7.9250,S,0
3,1,1,female,35.0,1,0,53.1000,S,1
4,0,3,male,35.0,0,0,8.0500,S,0


In [8]:
df = pd.get_dummies(df, columns=['sex','embarked','pclass'], dtype=int)

In [9]:
df.head()

,survived,age,sibsp,parch,fare,fam,sex_female,sex_male,embarked_C,embarked_Q,embarked_S,pclass_1,pclass_2,pclass_3
0,0,22.0,1,0,7.2500,1,0,1,0,0,1,0,0,1
1,1,38.0,1,0,71.2833,1,1,0,1,0,0,1,0,0
2,1,26.0,0,0,7.9250,0,1,0,0,0,1,0,0,1
3,1,35.0,1,0,53.1000,1,1,0,0,0,1,1,0,0
4,0,35.0,0,0,8.0500,0,0,1,0,0,1,0,0,1


In [10]:
y = df['survived']
X = df.drop('survived', axis=1)

In [12]:
all_columns = df.columns.to_list()
random_columns = [np.random.choice(all_columns, 3, replace=True).tolist() for i in range (10)]

In [13]:
random_columns

[['pclass_1', 'parch', 'survived'],
 ['age', 'sex_female', 'pclass_1'],
 ['fam', 'sibsp', 'parch'],
 ['sex_female', 'sex_female', 'fare'],
 ['embarked_Q', 'sex_male', 'parch'],
 ['pclass_2', 'embarked_Q', 'sex_male'],
 ['embarked_Q', 'pclass_2', 'sibsp'],
 ['pclass_3', 'embarked_C', 'age'],
 ['survived', 'sex_male', 'parch'],
 ['fam', 'embarked_C', 'survived']]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
def build_decision_trees(X_train, y_train, num_models=10, num_columns=3):
    all_columns = X_train.columns.to_list()
    models = []
    for i in range(num_models):
        selected_columns = np.random.choice(all_columns, num_columns, replace=True).tolist()
        X_train_subset = X_train[selected_columns]
        tree = DecisionTreeClassifier(random_state=42)
        tree.fit(X_train_subset, y_train)
        models.append((tree, selected_columns))
    return models

In [16]:
models = build_decision_trees(X_train, y_train, num_models=10, num_columns=3)

In [18]:
models

[(DecisionTreeClassifier(random_state=42),
  ['pclass_3', 'parch', 'embarked_Q']),
 (DecisionTreeClassifier(random_state=42), ['age', 'fare', 'embarked_C']),
 (DecisionTreeClassifier(random_state=42),
  ['embarked_C', 'embarked_C', 'fam']),
 (DecisionTreeClassifier(random_state=42), ['parch', 'sex_male', 'pclass_1']),
 (DecisionTreeClassifier(random_state=42), ['fam', 'embarked_C', 'sibsp']),
 (DecisionTreeClassifier(random_state=42),
  ['embarked_C', 'sex_male', 'pclass_3']),
 (DecisionTreeClassifier(random_state=42), ['sibsp', 'embarked_Q', 'fare']),
 (DecisionTreeClassifier(random_state=42), ['fam', 'pclass_3', 'embarked_C']),
 (DecisionTreeClassifier(random_state=42), ['sex_male', 'age', 'age']),
 (DecisionTreeClassifier(random_state=42),
  ['embarked_Q', 'sex_male', 'pclass_1'])]

In [19]:
def measure_accuracy(models, X_train, y_train):
    predictions = []
    for tree, selected_columns in models:
        X_train_subset = X_train[selected_columns]
        pred = tree.predict(X_train_subset)
        predictions.append(pred)
    predictions = np.array(predictions).T
    final_predictions, i = mode(predictions, axis=1)
    final_predictions = final_predictions.flatten()
    accuracy = accuracy_score(y_train, final_predictions)
    #print(predictions)
    #print(final_predictions)
    return accuracy
